# Diffusion用の前処理 発音時をチャンネル1, 伸びている状態をチャンネル2にする

In [1]:
import numpy as np
import pretty_midi
import pandas as pd
import collections
import glob
import os
import random
from tqdm import tqdm

In [2]:
def convert_midi(path, time_step, instrument_number, transpote_C, input_chord, chord, extention=1):
    """
    midiデータをond-hot-vectorに変換
    
    params
    ---------
    path: list
        midiデータのパス
    time_step: int
        分ける音符の種類
    instrument_number: int 
        midiファイルのインストルメント(トラック)数
    transpote_C: bool
        C調に移調するか
    input_chord: bool
        コードを入れるか
    return_chords: list
        コード（移調時に必要）
    extention: float
        テンポ120に合わせるときに伸ばす音の長さの倍数
    
    returns
    ---------
    midi_vector: list
        変換後のデータ(各配列はnumpy.ndarray)
    return_chords: list
        one-hot vectorに変換したコードデータ（移調しない場合は入力と変わらない）
    """
    #初期化
    midi = [] #各midiデータ
    tempo = np.zeros(len(path)) #テンポ
    midi_vector = [] #ベクトルに変換したmidiデータ
    return_chords = chord.copy()
    
    #midiデータ全体を格納
    for i in tqdm(range(len(path)), "data input"):
        midi.append(pretty_midi.PrettyMIDI(path[i]))
        tempo[i] = midi[i].get_tempo_changes()[1]
        #テンポを整数に四捨五入
        if tempo[i] - int(tempo[i]) >= 0.5:
            tempo[i] = int(tempo[i] + 1)
        else:
            tempo[i] = int(tempo[i])
                
    #分割せず変換する
    for i in tqdm(range(len(path)), desc='convert midi'):
        split = 60/(tempo[i]*time_step/4) #分割するためのパラメータ
        if extention != 1:
            split = split/extention
        #曲の長さを調べる
        length = []
        for n in range(instrument_number):
            melody_tmps = midi[i].instruments[n].notes
            vector_length = int(melody_tmps[-1].end/split) +1
            length.append(vector_length)
        vector_length = max(length) #曲の長さ
        #音を入れる
        midi_vector_tmp = [] #１曲分のベクトルに変換したデータ
        for n in range(instrument_number):
            #ベクトルに変換
            melody_tmps = midi[i].instruments[n].notes#音の情報の一時保存用のパラメータ
            melody_midi_tmp = np.zeros((2, vector_length, 88)) #分割前のベクトル
            for melody_tmp in melody_tmps:
                #音の情報
                start = int(melody_tmp.start/split)
                end = int(melody_tmp.end/split)
                if (melody_tmp.end/split)-end >= 0.5:
                    end += 1
                pitch = melody_tmp.pitch -21    
                
                #入力
                for k in range(start, end):
                    if k == start:
                        melody_midi_tmp[0][k][pitch] = 1
                    else:
                        melody_midi_tmp[1][k][pitch] = 1
            midi_vector_tmp.append(melody_midi_tmp)
        midi_vector.append(midi_vector_tmp)
    
    return midi_vector, return_chords
    

In [3]:
def convert_to_midi(vector, note_split):
    """
    midiに変換
    
    params
    -------------
    vector: numpy.ndarray
        変換するmany-hot vector(チャンネル数, 長さ, 88)
    note_split: int
        分割する音符の種類
    
    returns
    ---------
    piano: object
        midi_instrumentデータ
    """
    
    shapes = vector.shape
    instrument_name = pretty_midi.instrument_name_to_program('Acoustic Grand Piano')
    piano = pretty_midi.Instrument(program=instrument_name)

    #パラメータ
    start_time = 0.0
    end_time = 0.0
    tempo = 120
    time_split = 60/(tempo*note_split/4)

    #ノートを追加
    for p in range(shapes[-1]):
        start_time = 0.0
        end_time = 0.0
        durting = False #音が伸びているかどうか
        on = 0 #音が鳴っているかどうか
        for t in range(shapes[-2]):
            if not durting:
                on = int(round(vector[0][t][p]))
            end_time += time_split
    
            #次の時間において伸びているかの処理
            if on > 0:
                if t != shapes[-2]-1:
                    sutain = int(round(vector[1][t+1][p]))
                    if sutain > 0:
                        durting = True
                        continue
                    else:
                        durting = False
            else:
                start_time = end_time
                durting = False
                continue      
    
            #音符を追加
            note = pretty_midi.Note(
                velocity=70,
                pitch=int(p+21),
                start=start_time, 
                end=end_time
            )
            piano.notes.append(note)
            start_time = end_time
    
    return piano

In [4]:
#midiファイル名取得
path = ["001.mid"]
output_dir = "0000.mid"

#パラメータ
chord_dir = "chord_min_ver2.xlsx" #コードのファイル名（xlsxファイル)
input_chord = False #コード進行を入れるか
time_step = 16 #分割する音符
bar = 8 #何小節毎に分けるか
split_number = time_step*bar
instrument_number = 2 #midiファイルのインストルメント(トラック)数. (例）片手なら1, 右手左手を使うなら2
transpote_C = False #C調に移調するか
end_reduce = False #最後の音符を削除するか
observation_names = ['right', "left"] #保存する各observation(instrument)の名前

#各インストルメントの名前の数とトラック数が一致していなければ警告を出す
if len(observation_names) != instrument_number:
    print("WARING: 'instrument_number'と'obsevation_names'の数が一致していないため、正しくデータが入力されないです")

In [35]:
extention = 1 #テンポ120に合わせるときに伸ばす音の長さの倍数
#コードを読み込む
chords = []

#midiデータを読み込む
midi_tmp, chords = convert_midi(path, time_step, instrument_number, transpote_C, input_chord, chords, extention=extention)

midi = pretty_midi.PrettyMIDI()
    
#データ変換
instrument = []

for n in range(len(midi_tmp[0])):
    instrument.append(convert_to_midi(midi_tmp[0][n], time_step))
    
#instrument追加
for n in range(len(midi_tmp[0])):
    midi.instruments.append(instrument[n])
    
#書き込み
wirte_title = output_dir
midi.write(wirte_title)

convert midi: 100%|██████████| 1/1 [00:00<00:00, 122.19it/s]


In [19]:
midi_tmp

[[array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0.]],
  
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0.]]]),
  array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
     

In [28]:
len(file_title)

3

In [12]:
save_dict["text"]

'Generate Die_Lorelei, folk, C, major, innocent, happy, pure, and pop song.'